In [1]:
import scipy.optimize as sco
import pandas as pd 
import numpy as np 
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from tqdm import tqdm
from pulp import *
import plotly.express as px 
from sklearn.decomposition import PCA
import random
import plotly.io as pio 
pio.renderers.default = "browser"
import plotly.graph_objects as go
from pandas_datareader import data

In [13]:
    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/52))-1
        
        return(Result)

In [2]:
df = pd.read_csv("ETFs_info.csv", index_col="Ticker")
df["Inception"] = pd.to_datetime(df["Inception"])
df = df[df["Inception"] <= "2015-01-01"]

fundamental_df = pd.read_csv("fund_risk_cluster_reduced.csv",index_col="Ticker") #Fundamental Clustering Data
weekly_return = pd.read_csv("WeeklyReturns.csv",index_col="Date") #Weekly Return Data
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(weekly_return.columns)]
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(df.index)]
len(fundamental_df)

695

In [5]:
ticks = list(fundamental_df.index)

In [6]:

# startDate = "2015-01-01"
# endDate = "2021-01-01"
# # DOWNLOAD THE DATA FROM YAHOO DATABASE
# #------------------------------------------------------------------
# dailyPrices = data.DataReader(ticks, 'yahoo', startDate, endDate)
# dailyPrices = dailyPrices["Adj Close"]

In [7]:
# dailyPrices.to_csv("Dailyprices.csv", index="Date")
# daily_prices = pd.read_csv("DailyPrices.csv", index_col="Date")

In [3]:
def calculate_pdi( weekly_returns, portfolios, return_mean_range): 

    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for i,y in tqdm(zip(portfolios,range(1,len(portfolios)+1))):
        n_assets = len(i)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_ret  = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = weekly_returns[i].cov()
        port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(weekly_returns[i].corr().replace(np.nan, 0))
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[y] = {}
        PDI_dict[y]["PDI_INDEX"] = PDI
        PDI_dict[y]["# of Assets"] = len(i)
        PDI_dict[y]["Assets"] = i
        PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[y]["Annual Return"] = ann_ret
        PDI_dict[y]["Annual STD"] = port_std



    

        


    PDI_DF = pd.DataFrame(PDI_dict).T
    #PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
    PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
    PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
    PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
    PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
    PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)
    SPY_DF = PDI_DF.iloc[0,:]
    return PDI_DF,SPY_DF

In [17]:
def pca_per(return_data, initial_port, interval, ret_range_mean):

    data = return_data.copy() # data containing weekly returns
    tickers = list(data.columns)
    data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
    weeks_list = data.index # grabbing all index dates
    data.index = data.index.to_period(interval) # converting the index to quarterly sets
    periods = data.index.unique() # taking the unique quarters to loop

    
    print(periods)
    first_period = periods[0] # the first period of the time frame
    remaining_periods = periods[1:] # the remianing periods of the time framr
    first_periods = periods[:-1] # all periods minus the last

    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/ret_range_mean))-1
        
        return(Result)



    def optimal_weights(returns, port, lambda_l, range_period):


        def range_return_std(returns, port, weights):
            port_geo_ret = meanRetAn(list(returns[port].mul(weights,axis=1).sum(axis=1)))
            an_cov = returns[port].cov()
            port_std = np.sqrt(np.dot(weights.T, np.dot(an_cov, weights)))*np.sqrt(range_period)
            return  port_std,port_geo_ret




        def markowitz(weights, returns,lambda_l, port):
            p_var, p_ret = range_return_std(returns, port, weights)
            return -( (1- lambda_l)*p_ret - lambda_l*p_var )
            
        def markowitz_opt(returns, port,lambda_l):
            num_assets = len(port)
            args = (returns, lambda_l,port)
            constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            bound = (0.1,1)
            bounds = tuple(bound for asset in range(num_assets))
            result = sco.minimize(markowitz, num_assets*[1./num_assets,], args=args,
                                method='SLSQP', bounds=bounds, constraints=constraints)
            return result



        mark = markowitz_opt(returns, port, lambda_l).x



        return list(mark)



    def port_ret(returns,port, period): # function for calculating returns
        n_assets = len(initial_port)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_return = returns.loc[period][port].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        return  port_return


    
    def port_ret_w(returns,port,period,weights): # Calculating weightes returns
        port_return = returns.loc[period][port].mul(weights,axis=1).sum(axis=1)
        return  port_return



    performance_eq = []
    sharpe_performance_eq = []
    performance_w = []
    sharpe_performance_w = []
    weights = []
    weight_sr = []
    ############################################################ Calculate first period ####################################################################################
    #### Equal weight #####
    performance_eq.extend(port_ret(data, initial_port,first_period))
    sharpe_performance_eq.extend(port_ret(data, initial_port,first_period))
    tickers_weekly = list(data.columns)
    number_of_assets = [len(initial_port)]

    #### Weighted weight #####
    init_data = data.loc[first_period]
    ww = optimal_weights(init_data,initial_port,0.5,30)
    weights.append(ww)
    weight_sr.append(ww)
    performance_w.extend(port_ret_w(data, initial_port, first_period,ww))
    sharpe_performane_w.extend(port_ret_w(data, initial_port, first_period,ww))





    ################################################################## Portfolio Creation ###################################################################################
    samples = [["SPY"]]
    for number in [len(initial_port)]:
        for i in range(1,4000):
            #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
            samples.append(random.sample(list(tickers),number))
    samples_mini = []
    for i in samples:
        if i not in samples_mini:
            samples_mini.append(i)

    print("Number of Portfolios: {}".format(len(samples_mini)))

    vol_d = {}
    ######################################################## Calculation of portfolio perfomnce #############################################################################

    for init_time, next_time in zip(first_periods,remaining_periods):
        print("first time: {} - last time: {}".format(init_time, next_time))
        PDI_DF, SPY_DF = calculate_pdi(weekly_returns = data.loc[init_time].dropna(axis=1), portfolios = samples_mini, return_mean_range = 30)

        q1,q2 = PDI_DF["PDI_INDEX"].quantile([0.20,0.85])
        r1,r2 = PDI_DF["Annual Return"].quantile([0.4,0.85])


        #Taking the higest return within the range
        SPY_vol_lower = SPY_DF["Annual STD"] - 0.04 # allowing for a little room of divergence to find a asset match
        SPY_vol_upper = SPY_DF["Annual STD"] + 0.01# allowing for a little room of divergence to find a asset match

        PDI_rev = PDI_DF[(PDI_DF["Annual STD"] >= SPY_vol_lower) & (PDI_DF["Annual STD"] <= SPY_vol_upper) & (PDI_DF["PDI_INDEX"] >= q1) & (PDI_DF["PDI_INDEX"] <= q2)\
                         & (PDI_DF["Annual STD"] <= SPY_vol_upper) & (PDI_DF["Annual Return"] >= r1) & (PDI_DF["Annual Return"] <= r2)].reset_index().drop(columns="index")
                        # & (PDI_DF["PDI_INDEX"] >= q1) ].reset_index().drop(columns="index")


        # fig = px.scatter(PDI_DF,  color="Annual STD", y="Annual Return", x="PDI_INDEX")
        # fig.show()


        ################################################## Taking the higest return within the range ##########################################################################

        id = PDI_rev["Annual Return"].idxmax()
        port_max_ret = PDI_rev["Assets"][id]
        
        port_max_ret_period = port_ret(data,port_max_ret, next_time)
        performance_eq.extend(port_max_ret_period)

        ###### Weighted ###########
        period_return = data.loc[next_time]
        max_ww = optimal_weights(period_return,port_max_ret,0.5,30)
        weights.append(max_ww)
        port_max_ret_period_w = port_ret_w(data, port_max_ret,next_time,max_ww)
        performance_w.extend(port_max_ret_period)
    

        ################################################### Taking the best Sharpe Ratio within the range ######################################################################

        id_sharpe = PDI_rev["Sharpe Ratio"].idxmax()
        port_max_sharpe = PDI_rev["Assets"][id_sharpe]
        
        port_max_sharpe_period = port_ret(data,port_max_sharpe, next_time)
        sharpe_performance_eq.extend(port_max_sharpe_period)

        ###### Weighted ###########
        max_ww_sr = optimal_weights(period_return,port_max_sharpe,0.5,30)
        weight_sr.append(max_ww_sr)
        port_max_sharpe_period_w = port_ret_w(data, port_max_sharpe,next_time,max_ww_sr)
        sharpe_performance_w.extend(port_max_sharpe_period_w)






    dd = pd.DataFrame()
    dd["Time"] = weeks_list
    dd["per_eq"] = performance_eq
    dd["per_w"] = performance_w
    dd["per_weights"] = weights
    dd["SPY"] = list(weekly_return["SPY"])
    dd["sharpe_eq"] = sharpe_performance_eq
    dd["sharpe_w"] = sharpe_performance_w
    dd["sharpe_weights"] = weight_sr
    dd["per_eq_cum"] = dd["per_eq"].cumsum(axis=0)
    dd["per_w_cum"] = dd["per_w"].cumsum(axis=0)
    dd["SPY_cum"] = dd["SPY"].cumsum(axis=0)
    dd["sharpe_eq_cum"] = dd["sharpe_eq"].cumsum(axis=0)
    dd["sharpe_w_cum"] = dd["sharpe_w"].cumsum(axis=0)
    fig = px.line(dd, x="Time", y = ["per_eq_cum","SPY_cum","sharpe_eq_cum","sharpe_w_cum","per_w_cum"], title="Number of Assets: {} - period interval {}".format(len(initial_port), interval))




    return fig, dd




In [18]:
daily_prices = pd.read_csv("DailyPrices.csv", index_col="Date")

In [19]:
daily_prices = daily_prices.dropna(axis=1)

In [20]:
daily_prices = daily_prices.pct_change().dropna(axis=0)
daily_prices

,SPY,IVV,VTI,VOO,QQQ,VEA,IEFA,VWO,VUG,IEMG,...,IDMO,PXJ,FLM,FTLB,NFTY,FTAG,CHII,CHIM,CHIE,LTL
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.018059,-0.017559,-0.017183,-0.017569,-0.014669,-0.022287,-0.022459,-0.015925,-0.017458,-0.015239,...,0.000000,-0.041714,-0.025698,0.004931,0.003053,-0.013078,-0.014325,0.073320,0.000746,-0.032846
2015-01-06,-0.009419,-0.009010,-0.009798,-0.009833,-0.013408,-0.011126,-0.010376,-0.004880,-0.009470,-0.005231,...,-0.048465,-0.022063,-0.011823,-0.020609,-0.017156,0.013252,-0.020069,-0.012018,0.000000,-0.025528
2015-01-07,0.012461,0.012371,0.011933,0.012495,0.012891,0.010703,0.010298,0.022974,0.012517,0.020596,...,0.008554,-0.006707,0.003682,0.008016,0.010417,0.011066,0.012712,0.003841,-0.020880,0.001067
2015-01-08,0.017745,0.017864,0.017640,0.017569,0.019140,0.015205,0.012787,0.016906,0.017619,0.017819,...,0.000000,0.014733,0.013984,0.000000,0.027584,0.015921,-0.002789,0.003189,0.044174,0.023091
2015-01-09,-0.008013,-0.008438,-0.008290,-0.008421,-0.006583,-0.004814,-0.004575,-0.004963,-0.007461,-0.003586,...,-0.002699,-0.017544,-0.009722,0.009941,-0.008134,0.036239,-0.005594,0.000000,-0.015317,-0.013194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.003890,0.003877,0.001673,0.003996,0.004413,0.002572,0.000729,-0.005691,0.004522,-0.002810,...,-0.000908,-0.010101,0.002494,0.000742,0.011514,0.004255,0.002759,0.007075,0.002353,-0.003521
2020-12-28,0.008591,0.008695,0.006327,0.008698,0.010079,0.006201,0.007727,0.002044,0.008796,0.002984,...,0.004242,-0.010204,0.004729,-0.000988,0.012881,0.000000,0.015131,0.018735,0.030516,0.020697
2020-12-29,-0.001908,-0.001927,-0.004174,-0.002105,0.000895,0.007862,0.006944,0.010812,-0.002131,0.013058,...,0.009656,-0.006873,0.004297,-0.003462,0.004931,0.001541,-0.002033,-0.002299,-0.020501,-0.026459


In [21]:
ti = fundamental_df[(-fundamental_df["Category"].isin(['Trading--Leveraged Equity','Trading--Inverse Equity']) & (fundamental_df["Risk Cluster"].isin(['Moderat/High Risk - Slightly Above Market Volatility'])))]
tttt = list(ti.index)

daily_prices = daily_prices[daily_prices.columns.intersection(tttt)]

In [22]:
time, per, sharpe, fig , performance_frame= pca_per(daily_prices,['SPY','IVV','VTI','VOO'], interval="Q",ret_range_mean=30)
fig.show()

PeriodIndex(['2015Q1', '2015Q2', '2015Q3', '2015Q4', '2016Q1', '2016Q2',
             '2016Q3', '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4',
             '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1', '2019Q2',
             '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4'],
            dtype='period[Q-DEC]', name='Date', freq='Q-DEC')


NameError: name 'sharpe_performane_w' is not defined

In [495]:
#dddd = PDI_DF[PDI_DF["# of Assets"] == "14" ]
dddd = PDI_DF.copy()
#dddd = PDI_DF[PDI_DF["Annual STD"] <= 0.2 ]

In [496]:
dddd["PDI SEG"] = dddd["PDI_INDEX"].apply(lambda x: str(np.round(x, 1)))
dddd["STD SEGG"] = dddd["Annual STD"].apply(lambda x: np.round(x, 2))

In [497]:
dddd

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,PDI SEG,STD SEGG
1,NaN,1,[SPY],0.434199,0.041268,0.095045,nan,0.10
2,1.143835,3,"[GLIN, FDIS, URTH]",0.492642,0.063373,0.128638,1.1,0.13
3,1.403781,3,"[VDE, SPXL, EFG]",0.252745,0.045734,0.180947,1.4,0.18
4,1.193516,3,"[ENFR, FEX, IGN]",0.020559,0.002275,0.110633,1.2,0.11
5,1.206487,3,"[BICK, EWO, VEA]",0.260815,0.039640,0.151986,1.2,0.15
...,...,...,...,...,...,...,...,...
119991,1.985287,8,"[IAK, KCE, EEMA, FAS, ROBO, FPX, USD, IHDG]",0.244595,0.034916,0.142752,2.0,0.14
119992,2.584147,8,"[FLN, KFYP, HEFA, PSCC, PID, XSVM, QDEF, CXSE]",-0.181389,-0.023573,0.129960,2.6,0.13
119993,2.504402,8,"[VTWV, EXI, FDT, FXU, IHDG, KBWR, VONE, SCHC]",0.788838,0.077643,0.098427,2.5,0.10
119994,2.303937,8,"[IGE, VOOV, SDOG, ARKW, IEV, BFOR, VTWO, FCG]",0.186711,0.024852,0.133101,2.3,0.13


In [56]:
10*0.2

2.0

In [459]:
print("Min STD:", dddd["Annual STD"].min())
print("Max STD:", dddd["Annual STD"].max())

Min STD: 0.034345568498662105
Max STD: 0.4010084942247244


In [482]:
fig = px.bar(dddd, color="# of Assets", y="Annual Return", x="PDI_INDEX")
fig.show()

In [483]:
fig = px.scatter(dddd,  color="# of Assets", y="Annual Return", x="PDI_INDEX")
fig.show()

In [486]:
fig = px.box(dddd, x="# of Assets", y="PDI_INDEX")
fig.show()

In [498]:
fig = px.scatter_3d(dddd, x='STD SEGG', y = "Annual Return", z = "PDI_INDEX", color="# of Assets", hover_data = [dddd.index], title = "ETF Portfolios")
fig.show()

In [462]:
fig = px.scatter(dddd, x="PDI_INDEX", y="Annual Return", color = "STD SEG")
fig.show()

In [366]:
testest = dddd[dddd["# of Assets"] == "5"]
print(len(testest))
testest.head()

1999


,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
5994,1.044022,5,"[SQQQ, EMLP, IGM, EQL, IWB]",-0.584013,-0.042722,0.073153
5995,1.381763,5,"[QDF, FPX, IUSG, ICOL, FLN]",0.452782,0.094287,0.208239
5996,1.747769,5,"[FEUZ, QDF, VIOG, IGE, RWL]",0.370002,0.073438,0.198480
5997,2.163634,5,"[EDIV, SCHA, FCG, NXTG, EQL]",0.110250,0.024112,0.218707
5998,1.895498,5,"[EWMC, QTEC, EFA, FRAK, CHIE]",0.219108,0.048795,0.222700


## Without crypto's 

In [214]:
etf = pd.read_csv("WeeklyReturns.csv", index_col= "Date")
etf_name = list(etf.columns)

In [224]:
samples_etf = []

for number in range(2, 15,1):
    for i in range(1,5000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_etf.append(random.sample(etf_name,number))
samples_mini = []
for i in samples_etf:
    if i not in samples_mini:
        samples_mini.append(i)

print("Number of Portfolios: {}".format(len(samples_mini)))






Number of Portfolios: 64966


In [225]:
PDI_DF, SPY_DF = calculate_pdi(weekly_return, samples_mini, return_mean_range = 52)

64966it [03:01, 358.61it/s]


In [226]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',  color ="# of Assets", y= "Sharpe Ratio")
fig.show()

In [227]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',  color ="Annual STD", y= "Sharpe Ratio")
fig.show()

In [228]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',  y="Annual STD")
fig.show()

## Weigthing cryptoes in different sizes of a portfolio

In [299]:
crypto = pd.read_csv("crypto_weekly.csv",index_col="Date")
crypto_name = list(crypto.columns)
etf = pd.read_csv("WeeklyReturns.csv", index_col= "Date")
etf_name = list(etf.columns)
comb_df = pd.merge(crypto,etf, right_index=True,left_index=True)

In [300]:
ti = list(comb_df.columns)

samples = []
for number in [10,9,8,7,6,5,4,3,2,1]:
    for i in range(1,3000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples.append(random.sample(etf_name,number))
samples_mini = []
for i in samples:
    if i not in samples_mini:
        samples_mini.append(i)

print("Number of Portfolios: {}".format(len(samples_mini)))

#PDI_DF, SPY_DF = calculate_pdi(weekly_return, samples_mini, return_mean_range = 52)



Number of Portfolios: 27671


In [301]:
samples_mini[2997:3002]

[['SPDW', 'UAE', 'EZM', 'ACWX', 'DSI', 'IQDF', 'TDIV', 'PSQ', 'VEGI', 'QWLD'],
 ['ASHR', 'QTEC', 'MCHI', 'SMH', 'IDU', 'XLI', 'EQL', 'XNTK', 'IJH', 'CVY'],
 ['XLU', 'PGJ', 'SDOW', 'EZU', 'EDEN', 'XLB', 'QEFA', 'FDTS', 'DIM'],
 ['KBWR', 'XMMO', 'VO', 'PBS', 'JPXN', 'ACWI', 'PEY', 'IWV', 'SLX'],
 ['EFAV', 'SYG', 'VEGI', 'IDLV', 'VSS', 'XTL', 'IUSG', 'IYM', 'SCHC']]

In [302]:
per_cryp = []
for i in samples_mini:
    if int(len(i)) == 10:
        per = np.round(1-(int(len(i))/10),1)
        per_cryp.append(per)
    else:
        per = np.round(1-(int(len(i))/10),1)
        per_cryp.append(per)

In [303]:
per_cryp[-1]

0.9

In [310]:

new_port = []
for i in samples_mini:
    if len(i) == 10:
        new_port.append(i)
    else:
        new_port.append(i.extend(random.sample(crypto_name,10-int(len(i)))))

print("Number of Portfolios: {}".format(len(new_port)))

Number of Portfolios: 27671


In [311]:
new_port[2997:3002]

[['SPDW', 'UAE', 'EZM', 'ACWX', 'DSI', 'IQDF', 'TDIV', 'PSQ', 'VEGI', 'QWLD'],
 ['ASHR', 'QTEC', 'MCHI', 'SMH', 'IDU', 'XLI', 'EQL', 'XNTK', 'IJH', 'CVY'],
 ['XLU',
  'PGJ',
  'SDOW',
  'EZU',
  'EDEN',
  'XLB',
  'QEFA',
  'FDTS',
  'DIM',
  'DASH-USD'],
 ['KBWR', 'XMMO', 'VO', 'PBS', 'JPXN', 'ACWI', 'PEY', 'IWV', 'SLX', 'XVG-USD'],
 ['EFAV',
  'SYG',
  'VEGI',
  'IDLV',
  'VSS',
  'XTL',
  'IUSG',
  'IYM',
  'SCHC',
  'LTC-USD']]

In [306]:
PDI_DF, SPY_DF = calculate_pdi(comb_df, new_port, return_mean_range = 52)

2999it [00:09, 322.60it/s]


TypeError: object of type 'NoneType' has no len()

In [ ]:
PDI_DF["per_crypto"] = per_cryp # adding the percentage of cryptoes in the portfolio
PDI_DF["per_crypto"] = PDI_DF["per_crypto"].astype(str)

In [ ]:
PDI_DF

In [ ]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',  color ="per_crypto", y= "Sharpe Ratio")
fig.show()

## Markovitz